# Invasive Species with TensorFlow and SVM

Load important modules:

In [8]:
# load the necessary modules to start working on SVMs
from PIL import Image
from pandas import read_csv
import numpy as np
import tensorflow as tf

from tensorflow.python.framework import random_seed
from imgaug import augmenters as iaa
import imgaug as ia

def reset_graph(seed=1337):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
reset_graph()
DEFAULT_PATH = "D:/MIDS/W207_3 Machine Learning/project_data/"

Code for managing data sets:

In [9]:
# def load_images(indices, path = DEFAULT_PATH):
#     images= np.zeros(shape=(len(indices), 866, 1154, 3), dtype=np.uint8)
#     for p, i in enumerate(indices):
#         im = Image.open(path+str(i+1)+".jpg")
#         images[p] = np.array(im)
#     images = images.astype(np.float32)
#     images = np.multiply(images, 1.0 / 255.0)
#     return images

def load_images_flat(indices, labels, path = DEFAULT_PATH, seq = None):
    # resize to (400,300 from now and flatten array)
    # images= np.zeros(shape=(len(indices), 1440000), dtype=np.uint8)
    images= np.zeros(shape=(len(indices), 2998092), dtype=np.uint8)
    new_labels = np.zeros(shape = (labels.shape[0],))
    if seq:
        images= np.zeros(shape=(len(indices)*2, 2998092), dtype=np.uint8)
        new_labels = np.zeros(shape = (labels.shape[0]*2,))
    index = 0
    for p, i in enumerate(indices):
        im = Image.open(path+str(i+1)+".jpg")
        # doing an extra resize
        # arr = np.array(im.resize((800,600))).reshape(1,1440000)
        vanilla_arr = np.array(im).reshape(1,2998092)
        images[index] = vanilla_arr
        new_labels[index]=labels[p]
        if seq:
            aug_arr = seq.augment_images([np.array(im)])[0].reshape(1,2998092)
            index +=1
            images[index] = aug_arr
            new_labels[index]=labels[p]
        index +=1
    images = images.astype(np.float32)
    images = np.multiply(images, 1.0 / 255.0)
    return images, new_labels

class DataSet(object):

    def __init__(self,
                image_indices,
                labels,
                dataPath = DEFAULT_PATH,
                seed=None,
                aug_array = []):
        seed1, seed2 = random_seed.get_seed(seed)
        # If op level seed is not set, use whatever graph level seed is returned
        np.random.seed(seed1 if seed is None else seed2)
        self._num_examples = len(image_indices)
        self._image_indices = image_indices
        self._labels = labels
        self._epochs_completed = 0
        self._index_in_epoch = 0
        self.path = dataPath
        self.aug_array = aug_array

    @property
    def image_indices(self):
        return self._image_indices

    @property
    def labels(self):
        return self._labels

    @property
    def num_examples(self):
        return self._num_examples

    @property
    def epochs_completed(self):
        return self._epochs_completed

    def next_batch(self, batch_size, shuffle=True):
        """Return the next `batch_size` examples from this data set."""
        
        # pick augmentation sequence to apply.
        # applying an augmentation sequence means we will return twice the amount of batch_size items,
        # as each image sample will be read, and a copy of it will be augmented, making the samples 2*batch_size
        aug_seq = np.random.choice(self.aug_array, 1)[0] if self.aug_array else None
        
        start = self._index_in_epoch
        # Shuffle for the first epoch
        if self._epochs_completed == 0 and start == 0 and shuffle:
            perm0 = np.arange(self._num_examples)
            np.random.shuffle(perm0)
            self._image_indices = self.image_indices[perm0]
            self._labels = self.labels[perm0]
        # Go to the next epoch
        if start + batch_size > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            # Get the rest examples in this epoch
            rest_num_examples = self._num_examples - start
            images_rest_part = self._image_indices[start:self._num_examples]
            labels_rest_part = self._image_indices[start:self._num_examples]
            # Shuffle the data
            if shuffle:
                perm = np.arange(self._num_examples)
                np.random.shuffle(perm)
                self._image_indices = self.image_indices[perm]
                self._labels = self.labels[perm]
          # Start next epoch
            start = 0
            self._index_in_epoch = batch_size - rest_num_examples
            end = self._index_in_epoch
            images_new_part = self._image_indices[start:end]
            labels_new_part = self._labels[start:end]
            labels = np.concatenate((labels_rest_part, labels_new_part), axis=0)
            images, labels= load_images_flat(np.concatenate((images_rest_part, images_new_part), axis =0), labels, path = self.path, seq = aug_seq)
            return images, labels
        else:
            self._index_in_epoch += batch_size
            end = self._index_in_epoch
            labels = self._labels[start:end]
            images, labels = load_images_flat(self._image_indices[start:end], labels, path = self.path, seq = aug_seq)            
            return images, labels

Define Sequences of Image Augmentation and specify which one to use

In [10]:
# see https://github.com/aleju/imgaug
flip_lr_seq = iaa.Sequential([
    iaa.Fliplr(1), # horizontally flip
])

flip_ud_seq = iaa.Sequential([
    iaa.Flipud(1), # vertically flip
])

gauss_1_seq = iaa.Sequential([
    iaa.GaussianBlur(1.0), # blur images with a sigma of 0 to 3.0
])

flip_seq = iaa.Sequential([
    iaa.Flipud(1), # vertically flip
    iaa.Fliplr(1), # horizontally flip
])

flip_ud_gauss_seq = iaa.Sequential([
    iaa.Flipud(1), # vertically flip
    iaa.GaussianBlur(1.0),
])

flip_lr_gauss_seq = iaa.Sequential([
    iaa.Fliplr(1), # vertically flip
    iaa.GaussianBlur(1.0),
])

flip_2_gauss_seq = iaa.Sequential([
    iaa.Flipud(1), # vertically flip
    iaa.Fliplr(1), # horizontally flip
    iaa.GaussianBlur(1.0),
])
# pass this array to DataSet object. during next_batch() one of the seq in the array will be picked randomly and applied when loading data
seq_array = [flip_lr_seq, flip_ud_seq, gauss_1_seq, flip_seq, flip_ud_gauss_seq, flip_lr_gauss_seq, flip_2_gauss_seq]

In [21]:
# TESTING
print(np.random.choice(seq_array, 1))

im = Image.open("D:/MIDS/W207_3 Machine Learning/project_data/train/"+str(1)+".jpg")
seq = iaa.Sequential([
#     iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
    iaa.Fliplr(1), # horizontally flip 50% of the images
#     iaa.GaussianBlur(sigma=(0, 3.0)) # blur images with a sigma of 0 to 3.0
])
arr = np.array(im)
print(arr[:,0,:])
im_aug = seq.augment_images(arr)
print(im_aug[:,-1,:])
# arr = np.array(im.resize((1200,900))).reshape(1,1440000)
# arr = np.array(im.resize((1,2998092)))
# arr = np.array(im).reshape(1,2998092)
print(np.array_equal(arr,im_aug))
# TESTING

[ Sequential(name=UnnamedSequential, augmenters=[Flipud(name=UnnamedFlipud, parameters=[Binomial(Deterministic(float 1.00000000))], deterministic=False), GaussianBlur(name=UnnamedGaussianBlur, parameters=[Deterministic(float 1.00000000)], deterministic=False)], deterministic=False)]
[[ 32  73  33]
 [ 42  81  52]
 [ 61  87  76]
 ..., 
 [ 90 136 123]
 [101 154 136]
 [111 174 143]]
[[ 58  82  82]
 [ 62  80  81]
 [ 66  78  78]
 ..., 
 [127 157  86]
 [130 167 103]
 [108 158  80]]
False


Prepare train/test data and labels for working with tensorflow:

In [11]:
# prepare labels
train_labels = read_csv(DEFAULT_PATH + "train_labels.csv")['invasive'].values
y_train = train_labels[:1800]
# y_dev = train_labels[1800:]
y_validation = train_labels[1800:]

# prepare data
train_data = DataSet(np.arange(0,1800), y_train, dataPath = DEFAULT_PATH + "train/", aug_array = seq_array)
# test_data = DataSet(np.arange(1800,len(train_labels)), y_dev, dataPath = DEFAULT_PATH +"test/")
validation_data = DataSet(np.arange(1800,len(train_labels)), y_validation, dataPath = DEFAULT_PATH + "train/")


In [26]:
# TESTING
print(train_labels.shape[0]*2)
print(np.zeros(shape = (train_labels.shape[0]*2,)).shape)

4590
(4590,)


SVM Preparation: 

From my notes on choosing Kernel: Linear SVM (no kernel): Many training features, few training examples

In [12]:
svmC = 0.001
num_features = 2998092 #CHANGE WHEN RESIZING
n_epochs = 30
batch_size = 80
best_loss_val = np.infty
check_interval = 5
checks_since_last_progress = 0
max_checks_without_progress = 8
best_model_params = None 

SVM Variables:

In [13]:
# set graph
reset_graph()

# inputs
X = tf.placeholder(tf.float32, shape=[None, num_features], name="X")
y = tf.placeholder(tf.float32, shape=[None], name="y")
training = tf.placeholder_with_default(False, shape=[], name='training')

# set weights and create function
W = tf.Variable(tf.zeros([num_features,1]))
b = tf.Variable(tf.zeros([1]))
# y_raw = tf.matmul(X,W) + b
y_raw = tf.matmul(X,W) - b

# Optimization.
# regularization_loss = 0.5*tf.reduce_sum(tf.square(W))
regularization_loss = tf.reduce_sum(tf.square(W))
# hinge_loss = tf.reduce_sum(tf.maximum(tf.zeros([batch_size,1]), 1 - y*y_raw))

# HACK: *2 needed as the training involves augmentation, which adds each image again in an augmented form, so we get twice as big an array from load_image_flat
hinge_loss = tf.reduce_mean(tf.maximum(tf.zeros([batch_size*2,1]), 1 - y*y_raw))

# svm_loss = regularization_loss + svmC*hinge_loss
svm_loss = svmC*regularization_loss + hinge_loss
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(svm_loss)

# HACK: SVM Loss for validation set; TODO find a better way to do this
# This is needed because the training involves augmentation, which adds each image again in an augmented form
# This doesn't happen for validation
hinge_loss_test = tf.reduce_mean(tf.maximum(tf.zeros([batch_size,1]), 1 - y*y_raw))
svm_loss_test = svmC*regularization_loss + hinge_loss_test

# Evaluation.
predicted_class = tf.sign(y_raw);
correct_prediction = tf.equal(y,predicted_class)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

Define session and run

In [36]:
#TESTING
X_val, y_val = train_data.next_batch(80)
print(X_val.shape)
print(y_val.shape)
# with tf.Session() as sess:
#     init.run()
#     val = y_raw.eval(feed_dict={X: X_val, y: y_val})
#     print("accuracy")
#     acc_val = accuracy.eval(feed_dict={X: X_val, y: y_val})
#     print(acc_val)

(160, 2998092)
(160,)


In [ ]:
X_val, y_val = validation_data.next_batch(80)
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        # start an epock
        print("epoch = "+str(epoch))
        # print("W = "+str(W.eval()))
        for iteration in range(train_data.num_examples // batch_size):
            # get a batch to run
            X_batch, y_batch = train_data.next_batch(batch_size)
            # run the train
            # print(X_batch.shape)
            sess.run(train_step, feed_dict={X: X_batch, y: y_batch, training: True})
            loss_val = svm_loss_test.eval(feed_dict={X: X_val,y: y_val})
            if loss_val < best_loss_val:
                best_loss_val = loss_val
                checks_since_last_progress = 0
            else:
                checks_since_last_progress += 1
        # check accuracy
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_val, y: y_val})
        print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(epoch, acc_train * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            print()
            break

epoch = 0
Epoch 0, train accuracy: 66.2500%, valid. accuracy: 65.0000%, valid. best loss: 0.366802
epoch = 1
Epoch 1, train accuracy: 61.2500%, valid. accuracy: 65.0000%, valid. best loss: 0.366787
epoch = 2
